In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## doctors

In [ ]:
doctors = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/merge/doctors.csv')

In [ ]:
doctors

In [ ]:
del doctors['Unnamed: 0']

In [ ]:
doctors.info()

In [ ]:
doctors.describe()

посмотрим пропуски(функция с сема)

In [ ]:
#Воспользуемся следующей функцией, которая предоставляет отчет о пропусках в данных

def missing_values_table(df):
    """
    Функция возвращает резюме по пропущенным значениям
    """
    # Общее число пропусков
    mis_val = df.isnull().sum()

    # Процент пропусков
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Создадит таблицу с результатом
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Переименнуем колонки
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})

    # Отсортируем по проценту пропущенных значений
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)

    # Выведем некоторую информацию
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")

    return mis_val_table_ren_columns

missing_values_table(doctors)

# name (фио)

Всего в датасете - 38 865 строк

In [ ]:
doctors.shape[0]

Уникальных фио - 38441


In [ ]:
doctors.name.value_counts().size

230 фио повторяются более одного раза

In [ ]:
print((doctors.name.value_counts() > 1).sum())

Постмотрим на кол-во слов в фио

In [ ]:
doctors.name.str.split().apply(len).value_counts()

In [ ]:
# doctors.name.str.split().apply(len)[doctors.name.str.split().apply(len) == 5]

Встречаются лищние \t, но в основном:

- `в случае 2 слов` - это только фамилия имя
- `в случае 4 слов` - это дополнительная фамилия, чаще в скобках. например, фамилия до/после замужества. либо еще встречаются иностранные полные имена
- `в случае 5 слов` - либо несколько фамилий/отчеств, либо иностранные полныеимена

#experience

In [ ]:
def categorize_experience(exp):
    if pd.isna(exp):
        return 'No value'
    elif exp <= 5:
        return '0-5 лет'
    elif exp <= 10:
        return '6-10 лет'
    elif exp <= 15:
        return '11-15 лет'
    elif exp <= 20:
        return '16-20 лет'
    elif exp <= 30:
        return '21-30 лет'
    else:
        return 'Более 30 лет'

doctors['experience_category'] = doctors['experience'].apply(categorize_experience)
category_order = ['No value', '0-5 лет', '6-10 лет', '11-15 лет', '16-20 лет', '21-30 лет', 'Более 30 лет']

In [ ]:
doctors

распределение врачей по опыту работы

In [ ]:
plt.figure(figsize=(12, 8))

ax = sns.countplot(data=doctors, y='experience_category', order=category_order, palette='viridis', alpha=0.8)

total = len(doctors)
for p in ax.patches:
    width = p.get_width()
    percentage = f'{100 * width / total:.1f}%'
    x = p.get_x() + width
    y = p.get_y() + p.get_height() / 2
    ax.annotate(f'{int(width)}\n({percentage})', (x, y), ha='left', va='center', fontsize=10)

plt.title('распределение врачей по опыту работы')
plt.xlabel('количество врачей')
plt.ylabel('опыт работы')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(doctors['experience'],color='royalblue')

Найдем выбросы (выше конца уса / 3 IQR)

In [ ]:
q1 = doctors['experience'].quantile(0.25)
q3 = doctors['experience'].quantile(0.75)
iqr = q3 - q1

upper_whisker = q3 + 1.5 * iqr
upper_3iqr = q3 + 3 * iqr

print(upper_whisker, upper_3iqr, sep='\n')

Выше 3 IQR значений нет

In [ ]:
doctors[doctors['experience'] > upper_3iqr].shape[0]

Выше верхнего уса 18 значений

In [ ]:
doctors[doctors['experience'] > upper_whisker]['experience'].value_counts()

Врачи в пожилом возрасте с большим опытом также представлены на площадках

# Откуда врач? сбер или про докторов, сбер&продокторов

In [ ]:
def get_doctor_source(row):
    has_sber = pd.notna(row['link_sber'])
    has_prod = pd.notna(row['link_prod'])

    if has_sber and has_prod:
        return 'Обе платформы'
    elif has_sber:
        return 'SberHealth'
    elif has_prod:
        return 'ProDoctorov'

doctors['doctor_source'] = doctors.apply(get_doctor_source, axis=1)

In [ ]:
doctors

In [ ]:
doctors['doctor_source'].value_counts()

# experienceAndPlatform

In [ ]:
experienceAndPlatform = doctors.groupby('doctor_source')['experience'].agg(['mean','median','std','min','max'])
experienceAndPlatform

распределение опыта работы SberHealth/ProDoctorov

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].hist(doctors[(doctors['doctor_source']=='SberHealth')]['experience'],color='lightgreen')
axes[0].set_title('распределение опыта работы SberHealth')
axes[0].set_xlabel('опыт работы')
axes[0].set_ylabel('количество врачей')
axes[0].grid(alpha=0.3)

axes[1].hist(doctors[doctors['doctor_source']=='ProDoctorov']['experience'],color='cornflowerblue')
axes[1].set_title('распределение опыта работы ProDoctorov')
axes[1].set_xlabel('опыт работы')
axes[1].set_ylabel('количество врачей')
axes[1].grid(alpha=0.5)

#rating

In [ ]:
doctors['rating_sber'].describe()

In [ ]:
doctors['rating_prod'].describe()

распределение рейтингов SberHealth/ProDoctorov

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1,2,1)
sber_ratings = doctors[doctors['doctor_source'] == 'SberHealth']['rating_sber']
sns.histplot(sber_ratings, bins=20,alpha=0.7,color='lightgreen')
plt.title('распределение рейтингов SberHealth')
plt.xlabel('рейтинг')
plt.ylabel('количество врачей')
plt.xlim(0, 5.5)
plt.grid(alpha=0.3)

plt.subplot(1,2,2)
sber_ratings = doctors[doctors['doctor_source'] == 'ProDoctorov']['rating_prod'].dropna()
sns.histplot(sber_ratings, bins=20,alpha=0.7,color='cornflowerblue')
plt.title('распределение рейтингов ProDoctorov')
plt.xlabel('рейтинг')
plt.ylabel('количество врачей')
plt.xlim(0, 5.5)
plt.grid(alpha=0.3)

где рейтинг выше

In [ ]:
doctors_platforms_with_rating = doctors[
    (doctors['link_sber'].notna()) &
    (doctors['link_prod'].notna()) &
    (doctors['rating_sber'].notna()) &
    (doctors['rating_prod'].notna())].copy()

rating_diff = doctors_platforms_with_rating['rating_sber'] - doctors_platforms_with_rating['rating_prod']
higher_sber = (rating_diff > 0).sum()
higher_prod = (rating_diff < 0).sum()
equal = (rating_diff == 0).sum()

categories = ['выше на SberHealth', 'выше на ProDoctorov', 'равны']
values = [higher_sber, higher_prod, equal]
colors = ['lightgreen','cornflowerblue','bisque']
plt.pie(values, labels=categories, autopct='%1.1f%%',colors=colors)
plt.title('где рейтинг выше')

plt.tight_layout()
plt.show()

In [ ]:
doctors['rating_prod'].corr(doctors['rating_sber'])

corr оч низкая, никакая по факту

In [ ]:
sns.scatterplot(data=doctors, x='rating_prod', y='rating_sber',color = 'lightgreen')

зависимость рейтинга от стажа по платформам

In [ ]:
plt.figure(figsize=(12, 6))

plt.scatter(doctors[doctors['doctor_source']=='SberHealth']['experience'],
            doctors[doctors['doctor_source']=='SberHealth']['rating_sber'],alpha=0.6, color='lightgreen', s=60, label='SberHealth')

plt.scatter(doctors[doctors['doctor_source']=='ProDoctorov']['experience'],
            doctors[doctors['doctor_source']=='ProDoctorov']['rating_prod'],alpha=0.6, color='cornflowerblue', s=60, label='ProDoctorov')

plt.title('зависимость рейтинга от стажа по платформам')
plt.xlabel('опыт работы')
plt.ylabel('рейтинг')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
corr_sber = doctors[doctors['doctor_source']=='SberHealth']['experience'].corr(doctors[doctors['doctor_source']=='SberHealth']['rating_sber'])
corr_prod = doctors[doctors['doctor_source']=='ProDoctorov']['experience'].corr(doctors[doctors['doctor_source']=='ProDoctorov']['rating_prod'])
print(corr_sber,corr_prod)

# price

##основные характеристики

In [ ]:
doctors['price_sber'].describe()

In [ ]:
doctors['price_prod'].describe()

##выбросы

### sber

In [ ]:
Q1 = doctors['price_sber'].quantile(0.25)
Q3 = doctors['price_sber'].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR
print(lower_whisker,upper_whisker)

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors, x='price_sber',color = 'lightgreen')

рассмотрим без выбросов выше 3IQR

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors[doctors['price_sber'] < 3*IQR], x='price_sber',color='lightgreen')

### prodoctorov

In [ ]:
Q1 = doctors['price_prod'].quantile(0.25)
Q3 = doctors['price_prod'].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR
print(lower_whisker,upper_whisker)

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors, x='price_prod', color='cornflowerblue')

аналогично рассмотрим значение < 3IQR

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors[doctors['price_prod'] < 3*IQR], x='price_prod',color='cornflowerblue')

## cравнение цен для одних и тех же врачей на разных платформах

In [ ]:
plt.figure(figsize=(12, 6))

doctors_platforms_with_price = doctors[
    (doctors['link_sber'].notna()) &
    (doctors['link_prod'].notna()) &
    (doctors['price_sber'].notna()) &
    (doctors['price_prod'].notna())]

plt.scatter(doctors_platforms_with_price['price_sber'], doctors_platforms_with_price ['price_prod'], alpha=0.6, color='cornflowerblue', s=60)
plt.title('cравнение цен для одних и тех же врачей на разных платформах')
plt.xlabel('цена SberHealth')
plt.ylabel('цена ProDoctorov')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
corr = doctors_platforms_with_price['price_sber'].corr(doctors_platforms_with_price ['price_prod'])
corr

цены и правда +- такие же, если, они, конечно, есть

##добавление столбца price

*у нас высокая корреляция у двух столбцов(никакой не является целевой переменной), тогда сделаем общий столбец

price = (price_sber+price_prod)/2*

In [ ]:
doctors['price'] = (doctors['price_prod']+doctors['price_sber'])/2

In [ ]:
doctors['price'].describe()

In [ ]:
Q1 = doctors['price'].quantile(0.25)
Q3 = doctors['price'].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR
print(lower_whisker,upper_whisker)

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors, x='price',color = 'bisque')

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors[doctors['price'] < 3*IQR], x='price',color = 'bisque')

In [ ]:
plt.figure(figsize=(15, 5))

sns.histplot(doctors[(doctors['price'].notna()) & (doctors['price'] < 3*IQR)]['price'],bins=50, color = 'bisque',edgecolor='salmon')
plt.title('Распределение цен')
plt.xlabel('Цена')
plt.ylabel('Количество врачей')
plt.grid(alpha=0.3,axis='y')

##распределения цен на платформах

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
sns.histplot(doctors[(doctors['price_sber'].notna()) & ((doctors['price_sber'] < 3*IQR))]['price_sber'],
             bins=50, color = 'lightgreen',edgecolor='forestgreen')
plt.title('Распределение цен на SberHealth')
plt.xlabel('Цена')
plt.ylabel('Количество врачей')
plt.grid(alpha=0.3,axis='y')

plt.subplot(1, 2, 2)
sns.histplot(doctors[(doctors['price_prod'].notna()) & (doctors['price_prod'] < 3*IQR)]['price_prod'],
             bins=50,color = 'cornflowerblue',edgecolor='royalblue')
plt.title('Распределение цен на ProDoctorov')
plt.xlabel('Цена')
plt.ylabel('Количество врачей')
plt.grid(alpha=0.3,axis='y')

plt.tight_layout()
plt.show()

# ценовые категории для sber, prodoctorov, AvgPrice

In [ ]:
bins = [0, 1500, 3000, 5000, 7500, 10000, 15000, 25000, float('inf')]
labels = ['до 1.5k', '1.5k-3k', '3k-5k', '5k-7.5k', '7.5k-10k', '10k-15k', '15k-25k', '25k+']

doctors['price_category_sber'] = pd.cut(doctors['price_sber'], bins=bins, labels=labels)
doctors['price_category_prod'] = pd.cut(doctors['price_prod'], bins=bins, labels=labels)
doctors['price_category_avg'] = pd.cut(doctors['price'], bins=bins, labels=labels)

plt.figure(figsize=(12, 6))

sber_counts = doctors['price_category_sber'].value_counts().sort_index()
prod_counts = doctors['price_category_prod'].value_counts().sort_index()
price_counts = doctors['price_category_avg'].value_counts().sort_index()

x = np.arange(len(labels))
width = 0.35

bars1 = plt.bar(x - width/2, sber_counts, width, label='СберЗдоровье', alpha=0.7, color = 'lightgreen')
bars2 = plt.bar(x + width/2, prod_counts, width, label='ProDoctorov', alpha=0.7, color = 'cornflowerblue')
bars3 = plt.bar(x, price_counts, width, label='AvgPrice', color = 'slategrey')

for i, count in enumerate(sber_counts):
    plt.text(i-0.18, count + 20, str(count), ha='center')
for i, count in enumerate(prod_counts):
    plt.text(i+0.18, count + 20, str(count), ha='center')
for i, count in enumerate(price_counts):
    plt.text(i, count + 20, str(count), ha='center')

plt.xlabel('Ценовые категории')
plt.ylabel('Количество врачей')
plt.title('Врачи по ценовым категориям')
plt.xticks(x, labels)
plt.grid(alpha=0.3,axis='y')
plt.legend()
plt.tight_layout()
plt.show()


## зависимость цены от стажа по платформам

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(doctors[doctors['doctor_source']=='SberHealth']['experience'],
            doctors[doctors['doctor_source']=='SberHealth']['price_sber'], alpha=0.6, color='lightgreen', s=60, label='SberHealth')

plt.scatter(doctors[doctors['doctor_source']=='ProDoctorov']['experience'],
            doctors[doctors['doctor_source']=='ProDoctorov']['price_prod'], alpha=0.6, color='cornflowerblue', s=60, label='ProDoctorov')

plt.title('зависимость цены от стажа по платформам')
plt.xlabel('опыт работы')
plt.ylabel('цена')
plt.grid(alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
corr_sber = doctors[doctors['doctor_source']=='SberHealth']['experience'].corr(doctors[doctors['doctor_source']=='SberHealth']['price_sber'])
corr_prod = doctors[doctors['doctor_source']=='ProDoctorov']['experience'].corr(doctors[doctors['doctor_source']=='ProDoctorov']['price_prod'])
print(corr_sber,corr_prod)

нет такой зависимости тут

## сравнение средних цен по специальностям на разных платформах

In [ ]:
specialties = ['гинеколог', 'кардиолог', 'невролог', 'онколог', 'терапевт', 'эндокринолог']
avg_prices = pd.DataFrame(index=specialties, columns=['price_sber', 'price_prod'])

for specialty in specialties:
    specialty_doctors = doctors[doctors['speciality'].str.contains(specialty,na=False)]

    avg_sber = specialty_doctors['price_sber'].mean()
    avg_prices.loc[specialty, 'price_sber'] = avg_sber

    avg_prod = specialty_doctors['price_prod'].mean()
    avg_prices.loc[specialty, 'price_prod'] = avg_prod
avg_prices = avg_prices.astype(float)


plt.figure(figsize=(8, 6))
x = np.arange(len(specialties))
width = 0.35
plt.bar(x - width/2, avg_prices['price_sber'], width, label='SberHealth', alpha=0.8, color='lightgreen')
plt.bar(x + width/2, avg_prices['price_prod'], width, label='ProDoctorov', alpha=0.8, color='cornflowerblue')

plt.xlabel('Специальность')
plt.ylabel('Средняя цена')
plt.title('Сравнение средних цен по специальностям на разных платформах')
plt.xticks(x, specialties)
plt.legend()
plt.grid(axis='y', alpha=0.3)

for i in range(len(specialties)):
    sber_price = avg_prices['price_sber'].iloc[i]
    prod_price = avg_prices['price_prod'].iloc[i]
    plt.text(i - width/2, sber_price + 30, f'{sber_price:.0f}', ha='center', va='bottom', fontsize=9)
    plt.text(i + width/2, prod_price + 30, f'{prod_price:.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## посмотрим на разницы в ценах:

In [ ]:
price_diff = doctors_platforms_with_price['price_sber'] - doctors_platforms_with_price['price_prod']
price_diff.describe()

In [ ]:
doctors_with_price_diff = doctors.copy()
doctors_with_price_diff['price_diff'] = abs(doctors_platforms_with_price['price_sber'] - doctors_platforms_with_price['price_prod']) # по модулю
doctors_with_price_diff['price_diff'].describe()

среднее очень отличается от медианы

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors_with_price_diff, x='price_diff',color='royalblue')

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=doctors_with_price_diff, x='price_diff',color='royalblue')

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=doctors_with_price_diff, x='price_diff',color='royalblue')
plt.xlim(0, doctors_with_price_diff['price_diff'].quantile(0.97))

Очень много выбросов. Оценим по верхнему усу и 3 IQR

In [ ]:
q1 = doctors_with_price_diff['price_diff'].quantile(0.25)
q3 = doctors_with_price_diff['price_diff'].quantile(0.75)
iqr = q3 - q1

upper_whisker = q3 + 1.5 * iqr
upper_3iqr = q3 + 3 * iqr

print(upper_whisker, upper_3iqr, sep='\n')

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > upper_whisker].shape[0]

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > upper_3iqr].shape[0]

Посмотрим на те, где выбросы больше 3 IQR

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > upper_3iqr][['price_sber', 'price_prod', 'price_diff']]

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > upper_3iqr][['price_sber', 'price_prod', 'price_diff']].describe()

В этих случаях в осеовном выше на сбере

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > q3 + 9*iqr][['price_sber', 'price_prod', 'price_diff']]

Для самых высоких значений разниц цен - очень высокие цены на Продокторов, в 13 из 15 случаев (отклонения выше 9 IQR)

# где цена выше для одних и тех же враче

In [ ]:
higher_sber = (price_diff > 0).sum()
higher_prod = (price_diff < 0).sum()
equal = (price_diff == 0).sum()

categories = ['выше на SberHealth', 'выше на ProDoctorov', 'равны']
values = [higher_sber, higher_prod, equal]
colors = ['lightgreen','cornflowerblue','bisque']
plt.pie(values, labels=categories, autopct='%1.1f%%',colors=colors)
plt.title('где цена выше для одних и тех же врачей')

plt.tight_layout()
plt.show()

In [ ]:
corr = doctors_platforms_with_price['price_sber'].corr(doctors_platforms_with_price['price_prod'])
corr

#специальности

In [ ]:
doctors.columns

In [ ]:
def speciality_to_string(spec_list):
    if pd.isna(spec_list):
        return None
    if isinstance(spec_list, str):
        try:
            spec_list = eval(spec_list)
        except:
            return spec_list
    if isinstance(spec_list, list):
        return ', '.join(spec_list)
    return str(spec_list)
doctors['speciality'] = doctors['speciality'].apply(speciality_to_string)

In [ ]:
doctors['speciality']

In [ ]:
doctors['speciality'].unique()

In [ ]:
all_individual_specs = []
for spec_str in doctors['speciality'].dropna():
    specs = spec_str.split(', ')
    all_individual_specs.extend(specs)

individual_spec_counts = pd.Series(all_individual_specs).value_counts()

In [ ]:
individual_spec_counts.sample(10)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
top_25_specs = individual_spec_counts.head(25)
top_25_specs.plot(kind='barh', alpha=0.7,color='royalblue')
plt.title('частота топ-25 специальностей')
plt.xlabel('количество упоминаний')
plt.grid(axis='x', alpha=0.3)

for i, count in enumerate(top_25_specs.values):
    plt.text(count, i, str(count), va='center', fontsize=7)

plt.tight_layout()
plt.show()

## популярные специальности среди терапевтов

In [ ]:
doctors['speciality']

In [ ]:
terapevts = doctors[doctors['speciality'].str.contains('терапевт', case=False, na=False)]

terapevt_individual_specs = []
for spec_str in terapevts['speciality'].dropna():
    specs = spec_str.split(', ')
    terapevt_individual_specs.extend(specs)

terapevt_spec_counts = pd.Series(terapevt_individual_specs).value_counts()

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
top_25_specs = terapevt_spec_counts[1:26]
top_25_specs.plot(kind='barh', alpha=0.7,color='royalblue')
plt.title('частота топ-25 специальностей у терапевтов (помимо "терапевт")')
plt.xlabel('количество упоминаний')
plt.grid(axis='x', alpha=0.3)

for i, count in enumerate(top_25_specs.values):
    plt.text(count, i, str(count), va='center', fontsize=7)

plt.tight_layout()
plt.show()

# kids/adults

In [ ]:
plt.figure(figsize=(9, 6))

categories_count = {
    'only взрослые': doctors['is_adults'].sum(),
    'only дети': doctors['is_kids'].sum(),
    'и дети и взрослые': ((doctors['is_adults'] == True) & (doctors['is_kids'] == True)).sum(),
    'неть ничего': ((doctors['is_adults'] == False) & (doctors['is_kids'] == False)).sum()}

colors = sns.color_palette("viridis", len(categories_count))
plt.bar(categories_count.keys(), categories_count.values(),color=colors)
plt.title('распределение по категориям пациентов')
plt.ylabel('количество врачей')
plt.grid(axis='y', alpha=0.3)

for i, count in enumerate(categories_count.values()):
    plt.text(i, count, str(count), ha='center', va='bottom')

# metro

In [ ]:
plt.figure(figsize=(12,6))

all_metro_stations = []
metro_columns = [col for col in doctors.columns if 'metro' in col]
for col in metro_columns:
    metro_data = doctors[col].dropna()
    all_metro_stations.extend(metro_data)

metro_counts = pd.Series(all_metro_stations).value_counts()
top_20_metro = metro_counts.head(20)
top_20_metro.plot(kind='barh', color='royalblue', alpha=0.7)
plt.title('top-20 metro stations')
plt.xlabel('count clinics')
plt.grid(axis='x', alpha=0.3)

for i, count in enumerate(top_20_metro.values):
    plt.text(count, i, str(count), va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
metro_counts

# Число клиник (clinics_count)

In [ ]:
int(doctors['clinics_count_sber'].max())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
ax1, ax2 = axes

sns.histplot(data=doctors, x='clinics_count_sber', bins=int(doctors['clinics_count_sber'].max()), ax=ax1,color = 'lightgreen')
ax1.set_title('Распределение количества клиник (Сбер)')
ax1.set_xlabel('Количество клиник')
ax1.set_ylabel('Количество врачей')
ax1.set_ylim(0, 27559+1000)

sns.histplot(data=doctors, x='clinics_count_prod', bins=int(doctors['clinics_count_prod'].max()), ax=ax2,color = 'cornflowerblue')
ax2.set_title('Распределение количества клиник (Продокторов)')
ax2.set_xlabel('Количество клиник')
ax2.set_ylabel('Количество врачей')
ax2.set_ylim(0, 27559+1000)

plt.tight_layout()
plt.show()

In [ ]:
doctors['clinics_count_sber'].value_counts()

In [ ]:
doctors['clinics_count_prod'].value_counts()

Посмотрим среди тех, у кого клиники > 0

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
ax1, ax2 = axes

sns.histplot(data=doctors[doctors['clinics_count_sber']>0], x='clinics_count_sber', bins=int(doctors['clinics_count_sber'].max()), ax=ax1, color = 'lightgreen')
ax1.set_title('Распределение количества клиник (Сбер)')
ax1.set_xlabel('Количество клиник')
ax1.set_ylabel('Количество врачей')
ax1.set_ylim(0, 16986+1000)

sns.histplot(data=doctors[doctors['clinics_count_prod']>0], x='clinics_count_prod', bins=int(doctors['clinics_count_prod'].max()), ax=ax2,color = 'cornflowerblue')
ax2.set_title('Распределение количества клиник (Продокторов)')
ax2.set_xlabel('Количество клиник')
ax2.set_ylabel('Количество врачей')
ax2.set_ylim(0, 16986+1000)

plt.tight_layout()
plt.show()

In [ ]:
doctors['clinics_count_prod'].value_counts()

На Продокторов чаще есть информация о клиниках

# Heatmap

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(doctors.corr(numeric_only = True), annot=True, cmap='RdBu_r')